---
#Setup data files, py4cytoscape and Cytoscape connection
---
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

---
##Setup: Import source data files

In [1]:
# !rm GDS112_full.soft BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab
# !wget -q --no-check-certificate https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0
# !mv BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0 BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab
# !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
# !mv GDS112_full.soft?dl=0 GDS112_full.soft
!rm -r output/
!ls -l 
OUTPUT_DIR = 'output/'

rm: cannot remove 'output/': No such file or directory
total 4
drwxr-xr-x 1 root root 4096 Nov 13 17:33 sample_data


---
##Setup: Fetch latest py4cytoscape




**Note that you can fetch from a specific github branch by adding "@<branch>" to the "py4cytocape" at the end of the github URL.**

For example, to get branch 0.0.5: git+https://github.com/cytoscape/py4cytoscape@0.0.5

In [2]:
!pip uninstall -y py4cytoscape

!pip install py4cytoscape
#!pip install git+https://github.com/cytoscape/py4cytoscape@0.0.5
#!pip install git+https://github.com/cytoscape/py4cytoscape

     |████████████████████████████████| 143kB 8.9MB/s 
     |████████████████████████████████| 3.2MB 14.2MB/s 


---
##Setup: Set up Cytoscape connection


In [3]:
import IPython
import py4cytoscape as p4c
print(f'Loading Javascript client ... {p4c.get_browser_client_channel()} on {p4c.get_jupyter_bridge_url()}')
browser_client_js = p4c.get_browser_client_js(True)
IPython.display.Javascript(browser_client_js) # Start browser client


Loading Javascript client ... f9c54f76-724b-4445-9cc6-5f504e569593 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

---
#Sanity tests to verify Cytoscape connection


---
##Sanity test: Cytoscape version


In [4]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'automationAPIVersion': '1.0.0',
 'cytoscapeVersion': '3.8.2',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '0.0.6'}

---
## Sanity test: Cytoscape's sandbox path

**Sandboxing is explained in https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing**

In [5]:
p4c.sandbox_get_file_info('.')

{'filePath': 'C:\\Users\\CyDeveloper\\CytoscapeConfiguration\\filetransfer\\default_sandbox',
 'isFile': False,
 'modifiedTime': '2020-11-30 15:33:24.0470'}

In [11]:
gangsu_sandbox = p4c.sandbox_set('GangSu_sandbox', copy_samples=False, reinitialize=False)
gangsu_sandbox

'C:\\Users\\CyDeveloper\\CytoscapeConfiguration\\filetransfer\\GangSu_sandbox'

Get the gene expression data into a data frame that has Gene ID as a string ... that's what will fit with the 'name' column already in the BIOGRID network

ToDo:
1) Consider how to use read_csv to force Gene ID to string ... see here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html ... dtype parameter
2) *do* drop null GeneIDs
3) Consider how to use read_csv to avoid having to do wget in the first place

In [31]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
net_suid



In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/C:/Users/CyDeveloper/CytoscapeConfiguration/3/framework-cytoscape.log'}


1727796

In [5]:
import pandas as df
GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
GDS112_full.dropna(subset=['Gene ID'], inplace=True)
GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
print(GDS112_full.dtypes)


GDS112_full



ID_REF                    object
IDENTIFIER                object
GSM1029                  float64
GSM1030                  float64
GSM1032                  float64
GSM1033                  float64
GSM1034                  float64
Gene title                object
Gene symbol               object
Gene ID                   string
UniGene title            float64
UniGene symbol           float64
UniGene ID               float64
Nucleotide Title         float64
GI                       float64
GenBank Accession        float64
Platform_CLONEID         float64
Platform_ORF              object
Platform_SPOTID           object
Chromosome location      float64
Chromosome annotation     object
GO:Function               object
GO:Process                object
GO:Component              object
GO:Function ID            object
GO:Process ID             object
GO:Component ID           object
dtype: object


,ID_REF,IDENTIFIER,GSM1029,GSM1030,GSM1032,GSM1033,GSM1034,Gene title,Gene symbol,Gene ID,UniGene title,UniGene symbol,UniGene ID,Nucleotide Title,GI,GenBank Accession,Platform_CLONEID,Platform_ORF,Platform_SPOTID,Chromosome location,Chromosome annotation,GO:Function,GO:Process,GO:Component,GO:Function ID,GO:Process ID,GO:Component ID
24,25,TFC3,-0.663,0.144,0.605,0.696,0.659,transcription factor TFIIIC subunit TFC3,TFC3,851262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YAL001C,NaN,NaN,"Chromosome I, NC_001133.9 (147594..151166, com...","DNA binding///contributes_to DNA binding, bend...",5S class rRNA transcription from RNA polymeras...,mitochondrion///mitochondrion///colocalizes_wi...,GO:0003677///contributes_to GO:0008301///contr...,GO:0042791///GO:0042791///GO:0071168///GO:0006...,GO:0005739///GO:0005739///colocalizes_with GO:...
25,26,EFB1,0.678,0.343,0.844,-0.072,-0.084,translation elongation factor 1 subunit beta,EFB1,851260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YAL003W,NaN,NaN,"Chromosome I, NC_001133.9 (142174..143160)",guanyl-nucleotide exchange factor activity///t...,maintenance of translational fidelity///negati...,eukaryotic translation elongation factor 1 com...,GO:0005085///GO:0003746,GO:1990145///GO:0032232///GO:0006449///GO:0006...,GO:0005853///GO:0005853
26,27,SSA1,-0.956,-0.026,1.441,0.854,0.025,Hsp70 family ATPase SSA1,SSA1,851259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YAL005C,NaN,NaN,"Chromosome I, NC_001133.9 (139503..141431, com...",ATP binding///ATPase activity///nucleotide bin...,SRP-dependent cotranslational protein targetin...,cell wall///colocalizes_with chaperonin-contai...,GO:0005524///GO:0016887///GO:0000166///GO:0000...,GO:0006616///GO:0072318///GO:0002181///GO:0043...,GO:0005618///colocalizes_with GO:0005832///GO:...
27,28,FUN14,-0.435,-0.247,0.662,0.688,0.192,Fun14p,FUN14,851225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YAL008W,NaN,NaN,"Chromosome I, NC_001133.9 (136914..137510)",molecular_function,mitochondrion organization///phospholipid home...,integral component of membrane///integral comp...,GO:0003674,GO:0007005///GO:0055091,GO:0016021///GO:0031307///GO:0016020///GO:0005...
28,29,MDM10,-0.505,0.169,0.823,0.457,0.208,Mdm10p,MDM10,851223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YAL010C,NaN,NaN,"Chromosome I, NC_001133.9 (134184..135665, com...",molecular_function,establishment of mitochondrion localization///...,ERMES complex///ERMES complex///integral compo...,GO:0003674,GO:0051654///GO:0000002///GO:0070096///GO:0070...,GO:0032865///GO:0032865///GO:0016021///GO:0031...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9201,9202,VMA13,0.162,-0.202,0.649,0.132,0.042,H(+)-transporting V1 sector ATPase subunit H,VMA13,856148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YPR036W,NaN,NaN,"Chromosome XVI, NC_001148.4 (643836..645272)","hydrolase activity, acting on acid anhydrides,...",ATP hydrolysis coupled proton transport///ion ...,fungal-type vacuole membrane///integral compon...,GO:0016820///GO:0046961///GO:0046961,GO:0015991///GO:0006811///GO:0015992///GO:0006...,GO:0000329///GO:0016021///GO:0016020///GO:0005...
9203,9204,TIP41,0.084,-0.088,0.262,0.118,0.039,Tip41p,TIP41,856153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YPR040W,NaN,NaN,"Chromosome XVI, NC_001148.4 (647305..648375)",molecular_function,negative regulation of signal transduction///s...,cytoplasm///cytoplasm///nucleus///nucleus,GO:0003674,GO:0009968///GO:0007165///GO:0007165,GO:0005737///GO:0005737///GO:0005634///GO:0005634
9205,9206,ANT1,0.163,-0.314,0.011,0.350,0.472,Ant1p,ANT1,856246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YPR128C,NaN,NaN,"Chromosome XVI, NC_001148.4 (791218..792204, c...",adenine nucleotide transmembrane transporter a...,ATP transport///fatty acid beta-oxidation///fa...,cytoplasm///integral component of membrane///i...,GO:0000295///GO:0000295,GO:0015867///GO:0006635///GO:0006635///GO:0006...,GO:0005737///GO:0016021///GO:0016021///GO:0005...
9207,9208,RPS23B,0.849,0.124,-0.872,-1.023,-0.432,ribosomal 40S subunit protein S23B,RPS23B,856250,NaN,NaN,NaN,Na

In [6]:
p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

In commands_post(): java.lang.NullPointerException


CyError: ignored